# Illustration of mix-SQP solver applied to a small data set, and a large one

## Analysis setup

*Before attempting to run this Julia code, make sure your computer is properly set up to run this code by following the "Quick Start" instructions in the README of the [git repository](https://github.com/stephenslab/mixsqp-paper).*

We begin by loading the Distributions and LowRankApprox Julia packages, as well as some function definitions used in the code chunks below.

In [1]:
using Distributions
using LowRankApprox
include("../code/datasim.jl");
include("../code/likelihood.jl");
include("../code/mixSQP.jl");

Next, initialize the sequence of pseudorandom numbers.

In [2]:
srand(1);

## Generate a small data set

Let's start with a smaller example with 50,000 samples.

In [3]:
z = normtmixdatasim(round(Int,5e4));

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [4]:
sd = autoselectmixsd(z,nv = 20);
L  = normlikmatrix(z,sd = sd);
size(L)

(50000, 20)

## Fit mixture model using SQP algorithm 

First we run the mix-SQP algorithm once to precompile the function.

In [5]:
out = mixSQP(L,verbose = false);

Observe that only a small number of iterations is needed to converge to the solution of the constrained optimization problem.

In [6]:
k   = size(L,2);
x0  = ones(k)/k;
out = mixSQP(L,x = x0);

Running SQP algorithm with the following settings:
- 50000 x 20 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-08
- partial SVD max. error = 3.30e-07
iter      objective -min(g+1) #nnz #qp #ls
   1 3.03733620e+04 +6.30e-01   20   0   0
   2 2.09533189e+04 +5.80e+04    1   0   1
   3 1.28061712e+04 +2.01e+04    4   0   1
   4 1.11122668e+04 +8.74e+03    4   0   1
   5 1.09371560e+04 +4.16e+03    4   0   1
   6 1.07208708e+04 +2.05e+03    3   0   1
   7 1.05968491e+04 +1.05e+03    3   0   1
   8 1.05219786e+04 +5.20e+02    3   0   1
   9 1.03104419e+04 +2.57e+02    2   0   1
  10 1.01856260e+04 +1.31e+02    3   0   1
  11 1.01323900e+04 +6.64e+01    3   0   1
  12 1.00470781e+04 +3.29e+01    3   0   1
  13 9.90243273e+03 +1.64e+01    3   0   1
  14 9.85249949e+03 +8.20e+00    3   0   1
  15 9.81725606e+03 +3.95e+00    3   0   1
  16 9.77615717e+03 +1.86e+00    3   0   1
  17 9.75312640e+03 +8.52e-01    4   0   1
  18 9.7

## Generate a larger data set

Next, let's see what happens when we use the SQP algorithm to fit a mixture model to a much larger data set.

In [7]:
srand(1);
z = normtmixdatasim(round(Int,1e5));

## Compute the likelihood matrix

As before, we compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals. This time, we use a finer grid of $k = 40$ normal densities.

In [8]:
sd = autoselectmixsd(z,nv = 40);
L  = normlikmatrix(z,sd = sd);
size(L)

(100000, 40)

## Fit mixture model using SQP algorithm 

Even on this much larger data set, only a small number of iterations is needed to compute the solution.

In [9]:
k   = size(L,2);
x0  = ones(k)/k;
out = mixSQP(L,x = x0);

Running SQP algorithm with the following settings:
- 100000 x 40 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-08
- partial SVD max. error = 1.35e-06
iter      objective -min(g+1) #nnz #qp #ls
   1 6.21694207e+04 +6.60e-01   40   0   0
   2 4.83207934e+04 +3.93e-01   40   0   2
   3 3.75596772e+04 +2.25e-01   40   0   2
   4 2.99843226e+04 +1.23e-01   40   0   2
   5 2.13835273e+04 +7.20e+03    3   0   1
   6 1.98815668e+04 +2.53e+03    3   0   1
   7 1.97762416e+04 +1.12e+03    3   0   1
   8 1.97190698e+04 +5.21e+02    3   0   1
   9 1.96311484e+04 +2.54e+02    3   0   1
  10 1.95996287e+04 +1.25e+02    4   0   1
  11 1.95641608e+04 +6.15e+01    4   0   1
  12 1.95418659e+04 +3.09e+01    5   0   1
  13 1.95188614e+04 +1.54e+01    5   0   1
  14 1.95042433e+04 +7.72e+00    4   0   1
  15 1.94888513e+04 +3.80e+00    4   0   1
  16 1.94788438e+04 +1.87e+00    5   0   1
  17 1.94701551e+04 +8.56e-01    5   0   1
  18 1.

With no low-rank approximation (`lowrank = "none"`), the algorithm still converges even when using a very small correction factor `eps = 1e-12`.

In [10]:
out = mixSQP(L,x = x0,lowrank = "none",eps = 1e-12);

Running SQP algorithm with the following settings:
- 100000 x 40 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- Exact derivative computation (partial QR not used).
iter      objective -min(g+1) #nnz #qp #ls
   1 6.21694226e+04 +6.60e-01   40   0   0
   2 4.35976361e+04 +2.74e+08    2   0   1
   3 2.63754249e+04 +9.42e+07    3   0   1
   4 2.26716550e+04 +4.11e+07    3   0   1
   5 2.22369707e+04 +1.93e+07    3   0   1
   6 2.20822494e+04 +9.86e+06    3   0   1
   7 2.17850757e+04 +4.96e+06    3   0   1
   8 2.15686851e+04 +2.48e+06    2   0   1
   9 2.13181590e+04 +1.28e+06    3   0   1
  10 2.11397369e+04 +6.40e+05    2   0   1
  11 2.08812795e+04 +3.39e+05    3   0   1
  12 2.07941249e+04 +1.75e+05    3   0   1
  13 2.04915356e+04 +8.97e+04    3   0   1
  14 2.03990501e+04 +4.57e+04    3   0   1
  15 2.01797508e+04 +2.27e+04    3   0   1
  16 2.00663424e+04 +1.17e+04    3   0   1
  17 2.00068966e+04 +5.91e+03    3   0   1
  18 1.98269277e+04 +3.06

## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, and the versions of the Julia packages used here. 

In [11]:
Pkg.status("Distributions");
Pkg.status("LowRankApprox");
versioninfo();

 - Distributions                 0.15.0
 - LowRankApprox                 0.1.1
Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-7567U CPU @ 3.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Prescott)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, broadwell)
